In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

# Tutorial Code for SZ mode

In [7]:
## Initial Script
L = 8
N = 8
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

In [8]:
## Build Hamiltonian
t = 1
U = 2

b = driver.expr_builder()

# hopping term
b.add_term("cd", np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten(), -t)
b.add_term("CD", np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten(), -t)

# onsite term
b.add_term("cdCD", np.array([[i, ] * 4 for i in range(L)]).flatten(), U)

mpo = driver.get_mpo(b.finalize(), iprint=2)


Build MPO | Nsites =     8 | Nterms =         36 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     2 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     3 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     4 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.002
 Site =     5 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     6 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     7 /     8 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.011 Tmvc-total = 0.000 MPO bond dimension =     6 MaxDW = 0.00e+00
N

In [9]:
## Run DMRG
def run_dmrg(driver, mpo):
    ket = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)
    bond_dims = [250] * 4 + [500] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
        thrds=thrds, cutoff=0, iprint=1)

energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)


Sweep =    0 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.533 | E =      -6.2256341447 | DW = 7.19278e-16

Sweep =    1 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      1.026 | E =      -6.2256341447 | DE = -4.44e-15 | DW = 6.62658e-16

Sweep =    2 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      1.337 | E =      -6.2256341447 | DE = -5.33e-15 | DW = 4.07509e-16

Sweep =    3 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      1.976 | E =      -6.2256341447 | DE = -3.55e-15 | DW = 4.44966e-16

Sweep =    4 | Direction =  forward | Bond dimension =  500 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      3.312 | E =      -6.2256341447 | DE = -8.88e-15 | DW = 3.97501e-26

Sweep =    5 | Direction = backward 

# My Code for Bose Hubbard Model

In [11]:
## Initial Script
L = 9
N = 9
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SGB, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

In [17]:
L =4
np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten()

array([0, 1, 1, 0, 1, 2, 2, 1, 2, 3, 3, 2])

In [19]:
np.array([[i, ] * 4 for i in range(L)])

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3]])

In [20]:
t = 1
U = 1

b = driver.expr_builder()

# hopping term
b.add_term("cd", np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten(), -t)

# onsite term
b.add_term("cd", np.array([[i, ] * L for i in range(L)]).flatten(), U)

mpo = driver.get_mpo(b.finalize(), iprint=2)


Build MPO | Nsites =     9 | Nterms =         10 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     9 .. Mmpo =     4 DW = 0.00e+00

IndexError: invalid unordered_map<K, T> key

In [ ]:
# Parameters
L = 4  # Number of sites
t = 1
U = 1

# Initialize the DMRGDriver
driver = DMRGDriver()

# Create the expression builder
b = driver.expr_builder()

# Hopping term: Add hopping terms between neighboring sites
for i in range(L - 1):
    b.add_term("cd", [i, i + 1], -t)
    b.add_term("cd", [i + 1, i], -t)

# On-site interaction term: Add interaction terms at each site
for i in range(L):
    b.add_term("cd", [i, i], U)

# Finalize the expression builder
mpo = driver.get_mpo(b.finalize(), iprint=2)

# Check MPO dimensions and summary
if iprint:
    nnz, sz, bdim = mpo.get_summary()
    print(f"Non-zeros: {nnz}, Size: {sz}, Bond dimension: {bdim}")